## **05.a Evaluation of FlexZBoost: Computing Performance Metrics**
#### Authors: **Amanda Farias (afariassantos2@gmail.com), Iago Lopes (iagolops2012@gmail.com), Bruno Moraes (bruno.a.l.moraes@gmail.com)**,
#### Creation date: **07/15/2024**,  
#### Last Verifed to Run: **11/19/2024** (by @iago)

The objective of this notebook is to evaluate the performance of the machine learning model FlexZBoost, which was trained in *Notebook 3*. We utilize the test datasets defined in *Notebook 2* to assess the accuracy and reliability of the photo-z predictions. The evaluation includes computing metrics like $\sigma_z$, the fraction of catastrophic outliers ($FR_e$), and the bias ($\Delta z$) of the photo-z estimates. For the redshift distribution n(z), we compute the PIT, QQ plots, and  KS tests.
$~$
##### Logistics: This notebook is intended to be run through the Jupyter Lab NERSC interface available in __[Jupyter nersc](https://jupyter.nersc.gov/)__ in the **desc-python** kernel.

<div class="alert alert-block alert-danger">
<b>attention:</b> For the final notebook, we will use the test dataset from notebook 2, for now, let's use the previously output evaluated by Iago. </div>

### Importing packages used in this notebook

In [ ]:
import numpy as np
import pandas as pd
import tables_io
import qp
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm
from matplotlib.colors import LogNorm


from rail.core.data import (
    TableHandle,
    PqHandle,
    ModelHandle,
    QPHandle,
    DataHandle,
    Hdf5Handle,
    QPOrTableHandle
)
from rail.estimation.algos.flexzboost import FlexZBoostEstimator
from rail.estimation.algos.naive_stack import NaiveStackSummarizer
from rail.estimation.algos.true_nz import TrueNZHistogrammer
from rail.evaluation.dist_to_dist_evaluator import DistToDistEvaluator
from rail.evaluation.dist_to_point_evaluator import DistToPointEvaluator
from rail.evaluation.point_to_point_evaluator import PointToPointEvaluator
from rail.evaluation.single_evaluator import SingleEvaluator
from rail.core.stage import RailStage
from Metrics import plot_old_valid, plot_metrics, plot_scatter, plot_pit_qq

import warnings
warnings.filterwarnings("ignore")

<div class="alert alert-block alert-warning">
<b>ATTENTION:</b> This is a change you need to make to ensure the code works correctly, as it needs to run in your NERSC account.
</div> 

In [ ]:
nersc_name = 'iago'

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True
path= "/pscratch/sd/" + nersc_name[0] + "/" + nersc_name 
sigma = 10

## Loading results

In [ ]:
##############################################
############### Photo-z redshift #############
##############################################

############## Download the catalog with the photo-z #############

result = DS.read_file('pdfs_data', QPHandle, 
                      f'{path}/results_y1/output_estimate_a_roman_fzb_y1_{sigma}sig.hdf5')


############### Max redshift ##############
z_max = result().build_tables()['meta']['xvals'][0][-1]

############ X values for plot ############
zgrid = np.linspace(0, z_max, 301)

################## PDFS ##################
pdfs = result().build_tables()['data']['yvals']

########## Median or mode of PDF ##########
#mean = result().mean()
#mean = result().mode(zgrid)

#### Array of photo-z with 132891 galaxies
zphot = result().ancil['zmode'].flatten()

##############################################
################ True redshift ###############
##############################################

catalog = pd.read_csv(f'{path}/roman_rubin_y1_a_test_{sigma}sig.csv', sep=' ')
ztrue = catalog['redshift']
catalog['zphot'] = zphot

## Scatter

In [ ]:
plot_old_valid(zphot, ztrue, code='FlexZBoost', title = r'Point Estimates Scatter and KDE $N(z)$')

## Building the redshift distribution using Naive stack

The Naive stack method sums the PDFs of all galaxies without considering weights or other parameters. In the ``NaiveStackSummarizer`` from RAIL, the entire redshift range is divided into bins, and the PDFs of all galaxies within each bin are summed. Subsequently, the code normalizes these summed PDFs.

For our configuration, we utilize the redshift range defined by the output, starting from 0 to z_max, and divide this range into 300 bins.

For a more detailed explanation of each component in the ``NaiveStackSummarizer``, please refer to the __[RAIL documentation](https://github.com/LSSTDESC/rail/blob/6f4e15315962b3010dbd52eb2c4e308710df9b87/docs/source/new_rail_stage.rst#L90)__ on GitHub.

In [ ]:
##############################################
############### Spec-z redshift #############
##############################################

histogrammer = TrueNZHistogrammer.make_stage(nzbins=301,dz=0.01,zmax=z_max,zmin=0,hdf5_groupname='')
catal = catal_hdf.data['photometry']

def assign_tomo_bin(redshift):
    if 0 <= redshift < 3.1:
        return 0

catalog['class_id'] = catalog['redshift'].apply(assign_tomo_bin)

tomo_bins = catalog[['class_id']]

In [ ]:
catal = DS.read_file('ztrue_hdf', TableHandle, 
                      f'{path}/roman_rubin_y1_a_test_{sigma}sig.hdf5')

In [ ]:
nz_true = histogrammer.histogram(catal.data['photometry'],tomo_bins=tomo_bins)

In [ ]:
nz_true.data.plot(xlim=(0,3))
plt.title(r'$N(z_{true})$')

Applying this technique to the photo-zs.

In [ ]:
##############################################
############### Photo-z redshift #############
##############################################

stacker = NaiveStackSummarizer.make_stage(zmin=0.0, zmax=z_max, nzbins=301, nsamples = 20,name='phot',chunk_size=100000)
naive_results = stacker.summarize(result)

In [ ]:
nz_true = DS.read_file('nz_true',QPHandle,f'{path}/true_NZ.hdf5')
nz_stacked = DS.read_file('nz_phot',QPHandle,f'{path}/single_NZ_phot.hdf5')

In [ ]:
nz_phot = qp.read(f'{path}/single_NZ_phot.hdf5')
nz_true = qp.read(f'{path}/true_NZ.hdf5')

y_phot = nz_phot.objdata()['yvals'][0]
x_phot = nz_phot.metadata()['xvals'][0]

y_true = nz_true.objdata()['pdfs'][0]
x_true = nz_true.metadata()['bins'][0]

cs = UnivariateSpline(x_true[:-1], y_true)
cs.set_smoothing_factor(0.4) # adjust the smoothing of ztrue here !!!!

plt.figure(figsize=(10, 6))

plt.plot(x_true, cs(x_true), color='#0072B2', lw=2.5, label=r'$z_{\rm true}$', alpha=0.8)
plt.plot(x_phot, y_phot, color='#D55E00', lw=2.5, linestyle='--', label=r'$z_{\rm phot}$', alpha=0.8)

plt.xlabel('Redshift (z)', fontsize=18)
plt.ylabel('N(z)', fontsize=18)
plt.title(fr'$N(z)$ Naíve Stack {sigma}$\sigma$', fontsize=20)
plt.xlim(0,3)
plt.ylim(0,1.5)

plt.grid(True, linestyle='--', alpha=0.5)

plt.legend(fontsize=16)
plt.tight_layout()
plt.show()

## Visual metrics
In this segment, we will compute the following metrics: $\Delta z $, $\sigma_{68}$, $out_{3 \sigma}$, $out_{2 \sigma}$ e FR$_e$.

##### Definitions:
- $\Delta z$: The normalized difference between photometric and spectroscopic redshifts.

$\hspace{10cm} \Delta z = \frac{z_{phot} - z_{spec}}{1 + z_{spec}}$ 
<br>


- $\sigma_{68}$: The maximum absolute value of $\Delta z$ for the 68% of galaxies with the smallest $\Delta z$ values in bin $i$.

$\hspace{10cm} \sigma_{68} = max_{i \in U} \bigl\{ \big|\frac{z_{phot}^i - z_{spec}^i}{1 + z_{spec}^i} \big| \bigl\}$

where U is the set of the 68% of galaxies that have the smallest value of $|z_{phot} − z_{spec} |/(1+z_{spec})$ 
<br>

##### Outlier Metrics:
- $out_{3\sigma}$: The percentage of galaxies with $\Delta z$ greater than $3\sigma$.

- $out_{2\sigma}$: The percentage of galaxies with $\Delta z$ greater than $2\sigma$.




In [ ]:
plot_metrics(zphot,ztrue)

In [ ]:
from matplotlib.ticker import MaxNLocator

fig, axes = plt.subplots(3, 2, figsize=(10, 12), sharex=True, sharey=True)
plt.subplots_adjust(hspace=0.05, wspace=0.05)
random_i = [1117,17541,40009,46159,1429298,382]

for i, ax in enumerate(axes.flat[:6]):
    ax.plot(zgrid, pdfs[random_i[i]], color='dodgerblue', label='FlexZBoost')
    ax.axvline(ztrue[random_i[i]], color='orangered', label='z$_{true}$')

    ax.set_xlim(-0.05, 3.05)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True, prune='both'))  
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_yticks([])

    if i in [0, 2, 4]:
        ax.set_ylabel('p(z)', fontsize=16)
    if i in [4, 5]:
        ax.set_xlabel('redshift', fontsize=16)

    if i == 0:
        ax.legend(fontsize=16,loc=1)

plt.suptitle('PDF of a Individual Galaxy', fontsize=20, y=0.92)
plt.show()


In [ ]:
%%time
plot_pit_qq(pdfs,zgrid,z_true,title=fr'PIT QQ {sigma}$\sigma$',savefig=True)